[FROM HERE](https://github.com/drcrane/xvfb-alpine-docker)

In [1]:
# org Dockerfile from repo

```bash
# on Jan 26, 2019
cat <<EOF >Dockerfile_org_from_repo
FROM alpine:3.8

RUN apk update

RUN adduser -D -g users user
RUN mkdir -p /home/user \
	&& chown user:users /home/user

RUN apk add xvfb x11vnc fluxbox xdpyinfo st vim terminus-font \
	&& sed -r -i "s/\[exec\] \(xterm\) \{xterm\}/\[exec\] \(st\) \{st -f 'xos4 Terminus-14'\}/" /usr/share/fluxbox/menu \
	&& [[ ! -d /opt ]] && mkdir /opt \
	&& rm -vrf /var/cache/apk/*

COPY bootstrap.sh /opt

USER user
ENV HOME /home/user
WORKDIR /home/user
CMD ["/opt/bootstrap.sh"]
EOF

# build

In [ ]:
docker build --tag alpine-xvfb:beta1 Dockerfile_org_from_repo

# update alpine version

In [ ]:
```bash
# on Jan 26, 2019
cat <<EOF >Dockerfile
FROM alpine:3.18.5 

RUN apk update

RUN adduser -D -g users user
RUN mkdir -p /home/user \
	&& chown user:users /home/user

RUN apk add xvfb x11vnc fluxbox xdpyinfo st vim terminus-font \
	&& sed -r -i "s/\[exec\] \(xterm\) \{xterm\}/\[exec\] \(st\) \{st -f 'xos4 Terminus-14'\}/" /usr/share/fluxbox/menu \
	&& [[ ! -d /opt ]] && mkdir /opt \
	&& rm -vrf /var/cache/apk/*

COPY bootstrap.sh /opt

USER user
ENV HOME /home/user
WORKDIR /home/user
CMD ["/opt/bootstrap.sh"]
EOF

# build

In [ ]:
docker build --no-cache --tag alpine-xvfb:latest .

# run

In [ ]:
docker run --rm -it alpine-xvfb:latest

# docker container ip

In [ ]:
docker inspect -f \
'{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' <CONTAINER ID>
docker inspect -f \
'{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' ffc7f4bbc880

# 
export DISPLAY=:0
xhost +
vncviewer 172.17.0.2:0

# open shell inside container

In [ ]:
docker  exec -ti  ffc7f4bbc880 /bin/ash